In [193]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#	Select Important Features and create a dataframe 

In [194]:
data = pd.read_csv("healthcare-dataset-stroke-data.csv")
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [195]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()

In [196]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
data[['avg_glucose_level','bmi']]=scalar.fit_transform(data[['avg_glucose_level','bmi']])

In [197]:
data['gender'] = le.fit_transform(data['gender'])
data['ever_married'] = le.fit_transform(data['ever_married'])
data['work_type'] = le.fit_transform(data['work_type'])
data['Residence_type'] = le.fit_transform(data['Residence_type'])


In [198]:
values = OneHotEncoder().fit_transform( np.array(data['smoking_status']).reshape(-1, 1))
data['smoking_status']=values.toarray()

data['work_type']  = OneHotEncoder().fit_transform(np.array(data['work_type']).reshape(-1, 1)).toarray()


In [199]:
data['bmi']=data['bmi'].fillna(data['bmi'].median())
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

##	Split Data to train, test, validate 

In [200]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

## Train Model 

In [201]:

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=0)

In [202]:
y_pred = classifier.predict(X_test)
print(y_pred)

[0 0 0 ... 0 0 0]


In [203]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1212    1]
 [  65    0]]


0.9483568075117371

In [204]:
import pickle
with open('model_pkl', 'wb') as files:
    pickle.dump(classifier, files)

In [205]:
with open('model_pkl' , 'rb') as f:
    lr = pickle.load(f)
y_val_predict=lr.predict(X_test)